<center>
    <h1> INF285 - Computación Científica </h1>
    <h2> Tarea 4 - Código Base</h2>
    <h2> [S]cientific [C]omputing [T]eam </a> </h2>
    <h2> Version: 1.00</h2>
</center>

# Consideraciones

Asuma que se ingresaran *inputs* validos para las preguntas de desarrollo, es decir se puede prescindir u obviar las siguientes situaciones:
+ Integrales con límite inferior 1, no se ingresará algún límite superior menor a 1.
+ Al calcular beta(x,y) no se ingresarán pares de números $(x, y)$ tal que $x\leq0$ o $y\leq 0$.
+ No se ingresará un $n$ (puntos de Chebyshev) tal que la función colapse o genere overflow.
+ No se ingresará un $m$ grande tal que la función colapse o genere overflow.
+ No se ingresará $m<1$.

# No debe utilizar bibliotecas adicionales.

In [1]:
import scipy as sp
from scipy import integrate
from scipy import special
from scipy import interpolate
import numpy as np
import time

## Funciones base obligatorias

In [2]:
# La siguiente función calcula el valor de beta para un escalar 'x', 'y'.

# ATENCIÓN! Esta función es solo para el correcto funcionamiento
# de la siguiente función 'beta_repre', y NO debe ser llamada directamente.
def beta_def(x, y):
    inner_beta = lambda t,x,y: t**(x-1)*(1-t)**(y-1)
    return sp.integrate.quad(func=lambda t: inner_beta(t, x, y), a=0, b=1)[0]

# La siguiente función calcula el valor de beta para un vector X y un escalar y.

# ATENCIÓN! Cada vez que necesite calcular el valor de 'beta' para un input 'X', 'y',
# usted deberá llamar a la siguiente función:
"""
input
y      : (double) parametro fijo en beta_repre(x, y)
output
betas: (numpy array) valores de beta(X, y), 'X' un vector, 'y' un valor fijo.
"""
def beta_repre(y):    
    beta = np.vectorize(beta_def)
    return lambda X: beta(X, y)

```python
# Ejemplo de uso de beta_repre(y):
X=np.array([1,2,3,4])
beta = beta_repre(5)
print(beta(X))
>>>[0.2        0.03333333 0.00952381 0.00357143]
```

In [3]:
"""
input
xmin: (int) Valor minimo para el calculo
xmax: (int) Valor maximo para el calculo
n: (int) Cantidad de puntos de Chebyshev
output
return: (array) Puntos de Chebyshev
Esta funcion corresponde al GitHub oficial del curso por el profesor Claudio Torres
"""
def Chebyshev_points(xmin,xmax,n):
    ns = np.arange(1,n+1)
    x = np.cos((2*ns-1)*np.pi/(2*n))
    #y = np.sin((2*ns-1)*np.pi/(2*n))
    return (xmin+xmax)/2 + (xmax-xmin)*x/2


# Firma función 1
"""
input
func         : (function) función que se quiere integrar y debe estar vectorizada, esto significa 
                que si se le pasa un vector 'X' y otro vector 'Y', la función debe retornar la evaluación de 'func'
                para cada par  ('X_i', 'Y_i') (ya está implementado en beta_def, beta_repre).
a            : (double) límite inferior de integración
b            : (double) límite superior de integración
n            : (int) orden de cuadratura Gaussiana a utilizar por fixed_quad de scipy.integrate.
--------------
output
result       : (double) resultado de la integral entre a y b de la función func usando cuadratura Gaussiana de orden 'n'.
"""
def integral_killer(func, a, b, n):
    #Acá va su código(llamar a sp.integrate .... con func=func, a=a, b=b, n=n. [0])
    integral_result = sp.integrate.fixed_quad(func=func, a=a, b=b, n=n)[0]
    return integral_result

# Firma función 2

"""
input
b      : (double)   límite superior de integración
func   : (function) función beta_repre, ya viene por defecto, no modificar.
y           : (double) parámetro fijo en beta_repre(x, y)
n            : (int) orden de cuadratura Gaussiana a utilizar por fixed_quad de scipy.integrate.
----
output
result : (double) resultado de la integral de beta_repre(x, y) entre '1' y 'b'.
         hint:llamar a la función 1.
"""
def phi1(b, y, n, func=beta_repre):
    #HINT:to_integrate = func(y), en las siguientes funciones, la función beta a integrar también se define de esta forma.
    #Acá va su código
    to_integrate = func(y)
    result = integral_killer(to_integrate, 1, b, n)
    
    return result

# Función 3
"""
input
b           : (double) límite superior hasta donde se creará el polinomio interpolador
y           : (double) parámetro fijo en beta_repre(x, y)
chev        : (int) cantidad de puntos de Chebyshev a usar en el intervalo x \in [1, b]
func        : (function) función beta_repre, ya viene por defecto, no modificar.
output
interpolator: (callable) polinomio interpolador de beta_repre(x, y) en el intervalo x \in [1, b]
"""
def beta_interpolator_var(b, y, chev, func=beta_repre):
    # Acá va su código
    puntos = Chebyshev_points(1,b,chev)
    yi = func(y)(puntos)
    interpolator = interpolate.BarycentricInterpolator(puntos,yi)
    return interpolator

# Función 4
"""
input
chev        : (int) cantidad de puntos de Chebyshev a emplear en el intervalo x \in [1, 10]
y           : (double) parámetro fijo en beta_repre(x, y)
func   : (function) función beta_repre, ya viene por defecto, no modificar.
output
interpolator: (callable) polinomio interpolador de beta_repre(x, y) en x \in [1, 10]. y fijo.
"""
def beta_interpolator_fixed(chev, y, func=beta_repre):
    # Acá va su código
    puntos = Chebyshev_points(1,10,chev)
    yi = func(y)(puntos)
    interpolator = interpolate.BarycentricInterpolator(puntos,yi)
    return interpolator

# Función 5
"""
input
b           : (double) límite superior de integración
chev        : (int) cantidad de puntos de Chebyshev a considerar
y           : (double) parámetro fijo en beta_repre(x, y)
n            : (int) orden de cuadratura Gaussiana a utilizar por fixed_quad de scipy.integrate.
output
result      : (double) resultado de la integral del polinomio que interpola beta en [1, b]

HINT: interpolar con función 3 y luego usar integral_killer
"""
def phi2(b, y, chev, n):
    # Acá va su código
    funcion = beta_interpolator_var(b, y, chev)
    result = integral_killer(funcion, 1, b, n)
    return result

# Función 6
"""
input
inter: (calleable) polinomio interpolador de beta(x,y), x \in [1, 10], algún y fijo.
b : (double) límite superior de integración
n            : (int) orden de cuadratura Gaussiana a utilizar por fixed_quad de scipy.integrate.
output
results      : (double) resultado de la integral del polinomio que interpola beta x\in [1, 10], algun y fijo.

HINT: llamar integral_killer de inter, donde inter es el polinomio que ya viene creado desde afuera con beta_interpolator_fixed.
"""
def phi3(inter, b, n):
    # Acá va su código
    result = integral_killer(inter, 1, b, n)
    return result

```python
# Ejemplo de uso de integral_killer(func, a, b, n)
func = lambda x: x**2
resultado = integral_killer(func, 1, 3, 1e3)
print(resultado)
>>>8.666666666666398
# Lo cual corresponde a la integral entre 1 y 3 de x**2.
```
```python
# Ejemplo de uso para beta_interpolator_fixed(chev, y):
nums = np.array([1,2,3,4,5,6,7,8])
chev = 10
y = 5
polinomio_interpolador = beta_interpolator_fixed(chev, y)
print(polinomio_interpolador(nums))
>>>[1.97518508e-01 3.23039184e-02 1.02343962e-02 3.02134758e-03
 1.78634347e-03 9.58418583e-04 1.25105091e-04 5.08777247e-04]
```

## Funciones para testing

Las siguientes funciones ya estan implementadas y son para probar el código que se escribió.

In [4]:
"""
input
func_n: (int) corresponde a phi1 si func_n = 1, phi2 si func_n = 2, o phi3 si func_n = 3.
chev  : (int) cantidad de puntos de Chebyshev a considerar.
m : (int) cantidad de puntos equiespaciados a participar el intervalo [2, 10], límites inclusivos.
y           : (double) parametro fijo en beta_repre(x, y)
func   : (function) función beta_repre, ya viene por defecto, no modificar.
n            : (int) orden de cuadratura Gaussiana a utilizar por fixed_quad de scipy.integrate.
output
results      : (numpy array) resultados de la integral de func para los rangos dados.
"""
def test_betas(chev, m, y, func_n, n, func=beta_repre):
    if func_n == 1:
        results = np.array([phi1(b, y, n) for b in np.linspace(2, 10, m)])
    elif func_n == 2:
        results = np.array([phi2(b, y, chev, n) for b in np.linspace(2,10,m)])
    elif func_n == 3:
        polinom_fix = beta_interpolator_fixed(chev, y, func)
        results = np.array([phi3(polinom_fix, b, n) for b in np.linspace(2,10,m)])
    else:
        results = np.array([0])
    return results
"""
input
func_n : (int) corresponde a phi1 si func_1 = 1, phi2 si func_1 = 2, o phi3 si func_1 = 3.
chev  : (int) cantidad de puntos de Chebyshev a considerar, en caso de ser necesario.
m : (int) cantidad de puntos equiespaciados a participar el intervalo [2, 10], límites inclusivos.
y           : (double) parametro fijo en beta_repre(x, y)
n            : (int) orden de cuadratura Gaussiana a utilizar por fixed_quad de scipy.integrate.
output
tiempo      : (double) tiempo de computo de la llamada de func_n
"""
def timing(func_n, chev, m, y, n):
    if func_n < 1 or func_n > 3:
        tiempo = -1
    else:
        start = time.time()
        test_betas(chev, m, y, func_n, n)
        end = time.time()
        tiempo = end-start
    return tiempo
"""
input
chev  : (int) cantidad de puntos de Chebyshev a considerar, en caso de ser necesario.
m : (int) cantidad de puntos equiespaciados a participar el intervalo [2, 10], límites inclusivos.
y           : (double) parametro fijo en beta_repre(x, y)
n            : (int) orden de cuadratura Gaussiana a utilizar por fixed_quad de scipy.integrate.
output
winner      : (array) arreglo de métodos ordenado de más rápido a más lento
ordered_times: (array) con los tiempos de cómputo de los algoritmo implementados en la pregunta anterior, en el mismo orden que ganadores.
"""
def competencia(chev,m, y, n):
    times=np.zeros(3)
    times[0]=timing(1, chev, m, y, n)
    times[1]=timing(2, chev, m, y, n)
    times[2]=timing(3, chev, m, y, n)
    ordered_index=np.argsort(times)
    ordered_times=times[ordered_index]
    winner=ordered_index+1
    return winner, ordered_times

```python
# Ejemplo de uso de competencia:
ganadores, tiempos = competencia(100,20, 5, 1e3)
print(ganadores)
print(tiempos)
>>> ... *
>>> ... **
# Lo que se imprime en * es la lista en orden descendente donde el primer elemento es el mejor algoritmo.
#Lo que se imprime en ** es una lista con los tiempos que demoro en correr el algoritmo de la pregunta 6, en el mismo orden que ganadores.
```

## Referencias

- GitHub oficial del curso por el profesor Claudio Torres.

https://github.com/tclaudioe/Scientific-Computing/blob/master/SC1v2/05_Polynomial_Interpolation_1D.ipynb